# Homework1

## Import libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
tfk = tf.keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, Resizing, Rescaling

## Random seed for reproducibilty

In [ ]:
seed = 1234
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Data Load and Augmentation
images were resized to 139x139 because, during development, we wrongly read that for inceptionV3 "width and height should be no smaller than 139" but actually the keras documentation says "width and height should be no smaller than 75"

In [ ]:
batch_size=32
validation_split=0.2
data_dir = "../input/dataset/training_data_final"

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rotation_range=90,
                                   preprocessing_function=preprocess_input,
                                   brightness_range=(0.65, 1.35),
                                   height_shift_range=5,
                                   width_shift_range=5,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   vertical_flip=True, 
                                   fill_mode='reflect',
                                   validation_split=validation_split)                                 

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        validation_split=validation_split)

train_data = train_datagen.flow_from_directory(directory=data_dir,
                                               target_size=(139,139), 
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               classes=None,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training',
                                               seed=seed
                                              ) 

validation_data = validation_datagen.flow_from_directory(directory=data_dir,
                                                        target_size=(139,139), 
                                                        color_mode='rgb',
                                                        batch_size=batch_size,
                                                        classes=None,
                                                        class_mode='categorical',
                                                        shuffle=True,
                                                        subset = 'validation',
                                                        seed=seed
                                                        )

## Preview some samples

In [ ]:
def get_next_batch(generator):
  batch = next(generator)

  image = batch[0]
  target = batch[1]

  print("(Input) image shape:", image.shape)
  print("Target shape:",target.shape)

  # Visualize only the first sample
  image = image[0]*255
  target = target[0]
  target_idx = np.argmax(target)
  print()
  print("Categorical label:", target)
  print("Label:", target_idx + 1)
  print("Class name:", target_idx + 1)
  fig = plt.figure(figsize=(6, 4))
  plt.imshow(np.uint8(image))

  return batch


In [ ]:
trainImage = get_next_batch(train_data)[0]
validationImage = get_next_batch(validation_data)[0]

## Transfer Learning

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(include_top=False,
                         weights="imagenet",
                         input_shape=(139,139,3))

In [ ]:
tl_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256,
          activation='relu'),
    Dropout(0.5),
    Dense(256,
          activation='relu'),
    Dropout(0.4),
    Dense(8, activation='softmax')
])

In [ ]:
tl_model.summary()

In [ ]:
# Freeze the first layers until 288th
tl_model.get_layer('inception_v3').trainable = True
for i, layer in enumerate(tl_model.get_layer('inception_v3').layers[:288]):
  layer.trainable=False

In [ ]:
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(),
              optimizer=tfk.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
# setting the weight of the classes in order to manage the problem of imbalanced dataset
class_weight = {0 : 2.9,
                1 : 1.0,
                2 : 1.0,
                3 : 1.0,
                4 : 1.0,
                5 : 2.4,
                6 : 1.0,
                7 : 1.0}

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

In [ ]:
tl_history = tl_model.fit(x= train_data,
                    validation_data= validation_data,
                    epochs= 200,
                    batch_size=batch_size,
                    class_weight = class_weight,
                    callbacks = [early_stopping]
                   ).history

In [ ]:
tl_model.save('tl_model')

In [ ]:
# Plot the training
plt.figure(figsize=(20,5))
plt.plot(tl_history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(20,5))
plt.plot(tl_history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
del tl_model

## Partial Fine Tuning

In [ ]:
ft_model = tfk.models.load_model('tl_model')

In [ ]:
# Set all InceptionV3 layers to True
ft_model.get_layer('inception_v3').trainable = True

In [ ]:
# Freeze the first layers until 215th
for i, layer in enumerate(ft_model.get_layer('inception_v3').layers[:215]):
  layer.trainable=False

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(),
                 optimizer=tfk.optimizers.Adam(8e-4),
                 metrics=['accuracy'])

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = train_data,
    validation_data = validation_data,
    batch_size = batch_size,
    class_weight = class_weight,
    epochs = 100,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)]
).history

In [ ]:
ft_model.save('ft_model')

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')

plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')

plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')

plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')

plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
del ft_model

## Full Fine Tuning

In [ ]:
full_model = tfk.models.load_model('ft_model')

In [ ]:
# Set all InceptionV3 layers to True
full_model.get_layer('inception_v3').trainable = True

In [ ]:
full_model.summary()

In [ ]:
# Compile the model
full_model.compile(loss=tfk.losses.CategoricalCrossentropy(),
                 optimizer=tfk.optimizers.Adam(4e-5),
                 metrics=['accuracy'])

In [ ]:
# Fine-tune the model
full_history = full_model.fit(
    x = train_data,
    batch_size = batch_size,
    epochs = 40,
    validation_data = validation_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)]
).history

In [ ]:
full_model.save('full_model')

## Final Plots

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))


plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')

plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')

plt.plot(full_history['loss'], alpha=.3, color='#FFA500', linestyle='--')
plt.plot(full_history['val_loss'], label='Full Fine Tuning', alpha=.8, color='#FFA500')


plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))


plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')

plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')

plt.plot(full_history['accuracy'], alpha=.3, color='#FFA500', linestyle='--')
plt.plot(full_history['val_accuracy'], label='Full Fine Tuning', alpha=.8, color='#FFA500')


plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()